In [31]:
import codecs
import os
import sys
import zipfile
import pathlib

import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
import tensorflow_datasets as tfds

In [32]:
(train_ds, test_ds), metadata = tfds.load('colorectal_histology',split=['train[:80%]','train[80%:]'],with_info=True,
        as_supervised=True)

In [33]:
class CNN(tf.keras.Sequential):
    def __init__(self):
        super(CNN, self).__init__(layers=[
            tf.keras.layers.Rescaling(1./255, input_shape=(150,150,3)), # input length = 150
            tf.keras.layers.Conv2D(32, 5, activation='relu'), # output length = input length - 4 = 146
            tf.keras.layers.MaxPooling2D(), # pool_size = (2,2) -> output length = input length / 2 = 73
            tf.keras.layers.Conv2D(32, 5, activation='relu'), # output length = 69
            tf.keras.layers.MaxPooling2D(), # output length = 34
            tf.keras.layers.Conv2D(32, 5, activation='relu'), # output length = 30
            tf.keras.layers.MaxPooling2D(), # output length = 15
            tf.keras.layers.Flatten(), # output length = 32 * 15 * 3 = 1440
            tf.keras.layers.Dense(7200, activation='relu'),
            tf.keras.layers.Dense(8)
        ])

In [34]:
model = CNN()

optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)

criterion = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

In [35]:
model.compile(optimizer=optimizer, loss=criterion, metrics=['accuracy'])

In [36]:
model.summary()

Model: "cnn_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 rescaling_6 (Rescaling)     (None, 150, 150, 3)       0         
                                                                 
 conv2d_18 (Conv2D)          (None, 146, 146, 32)      2432      
                                                                 
 max_pooling2d_18 (MaxPoolin  (None, 73, 73, 32)       0         
 g2D)                                                            
                                                                 
 conv2d_19 (Conv2D)          (None, 69, 69, 32)        25632     
                                                                 
 max_pooling2d_19 (MaxPoolin  (None, 34, 34, 32)       0         
 g2D)                                                            
                                                                 
 conv2d_20 (Conv2D)          (None, 30, 30, 32)        25632 

In [37]:
15*15*32

7200

In [39]:
model.fit(
  train_ds.batch(1),
  batch_size=1,
  epochs=1
)

ValueError: Creating variables on a non-first call to a function decorated with tf.function.

In [ ]:
model.evaluate(test_ds, verbose=2)